# Input module examples

This is not yet perfect, but is getting there. The dataframes which are returned are designed to reflect MAGICC's underlying data input format. When we have pint with units (waiting on https://github.com/hgrecco/pint/pull/684) we can use `df.quantify()` and `df.dequantify()` to quickly add/remove units to our dataframes and swiftly do conversions etc. 

In [1]:
from os.path import join
from pprint import pprint

import pandas as pd
import pymagicc
from pymagicc.input import MAGICCInput

import expectexception

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
MAGICC6_DIR = join("..", "pymagicc", "MAGICC6", "run")

In [3]:
"""
temporary sandbox: delete later

DELETE
DELETE
DELETEDELETEDELETE
DELETE
DELETE
DELETE
DELETE
DELETE
"""
mdata = MAGICCInput()
mdata.read(MAGICC6_DIR, "HISTRCP_HFC43-10_CONC.IN")
mdata.df

VARIABLE,HFC4310_CONC
TODO,SET
UNITS,Parts per trillion (ppt)
REGION,GLOBAL_MIXINGRATIO
YEAR,
1930,0.0
1931,0.0
1932,0.0
1933,0.0
1934,0.0
1935,0.0


## Read files

In [4]:
mdata = MAGICCInput()
mdata.read(MAGICC6_DIR, "HISTRCP_CO2I_EMIS.IN")
mdata.head()

VARIABLE CO2I_EMIS                         
TODO           SET                         
UNITS          GtC                         
REGION      R5OECD R5REF R5ASIA R5MAF R5LAM
YEAR                                       
1765         0.003   0.0    0.0   0.0   0.0
1766         0.003   0.0    0.0   0.0   0.0
1767         0.003   0.0    0.0   0.0   0.0
1768         0.003   0.0    0.0   0.0   0.0
1769         0.003   0.0    0.0   0.0   0.0

In [5]:
pprint(mdata.metadata)

{'contact': 'Base year emissions inventories: Steve Smith (ssmith@pnl.gov) and '
            'Jean-Francois Lamarque (Jean-Francois.Lamarque@noaa.gov); RCP '
            '3-PD (IMAGE): Detlef van Vuuren (detlef.vanvuuren@pbl.nl); RCP '
            '4.5 (MiniCAM): Allison Thomson (Allison.Thomson@pnl.gov); RCP 6.0 '
            '(AIM): Toshihiko Masui (masui@nies.go.jp); RCP 8.5 (MESSAGE): '
            'Keywan Riahi (riahi@iiasa.ac.at); Concentrations & Forcing '
            'compilation: Malte Meinshausen (malte.meinshausen@pik-potsdam.de)',
 'data': 'Historical fossil&industrial CO2 (CO2I) Emissions '
         '(HISTRCP_CO2I_EMIS)',
 'header': 'RCPTOOL - MAGICC 6.X DATA FILE \r\n'
           'VERSION:   ALPHA - FIRST DRAFT - PRIMAP xls file written on 02 Sep '
           '2009, 17:04:37 \r\n'
           ' \r\n'
           ' \r\n'
           'DATA:  Historical fossil&industrial CO2 (CO2I) Emissions '
           '(HISTRCP_CO2I_EMIS) \r\n'
           'SOURCE:  RCP data as presented on '

## Accessors

`MAGICCInput` is built to make data accessing as simple as possible. As we have unique keys for each of our headers, we allow users to select data simply using lists, without specifying what each value should correspond to etc. (essentially we hide away all the pandas fiddly stuff).

Of course the underlying dataframe is always accessible via the `.df` attribute.

In [6]:
mdata['CO2I_EMIS'].head(2)

VARIABLE CO2I_EMIS                         
TODO           SET                         
UNITS          GtC                         
REGION      R5OECD R5REF R5ASIA R5MAF R5LAM
YEAR                                       
1765         0.003   0.0    0.0   0.0   0.0
1766         0.003   0.0    0.0   0.0   0.0

In [7]:
mdata['R5ASIA', 'CO2I_EMIS'].head(2)

VARIABLE,CO2I_EMIS
TODO,SET
UNITS,GtC
REGION,R5ASIA
YEAR,
1765,0.0
1766,0.0


In [8]:
# changing order doesn't matter
mdata['CO2I_EMIS', 'R5ASIA'].head(2)

VARIABLE,CO2I_EMIS
TODO,SET
UNITS,GtC
REGION,R5ASIA
YEAR,
1765,0.0
1766,0.0


In [9]:
%%expect_exception KeyError
# the above doesn't work on the raw dataframe
# because you need to specify all the levels 
# in the right order
mdata.df['R5ASIA', 'CO2I_EMIS']

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-9-b7d9f3057b51> in <module>()
      2 # because you need to specify all the levels
      3 # in the right order
----> 4 mdata.df['R5ASIA', 'CO2I_EMIS']

~/Documents/AGCEC/MCastle/pymagicc/venv/lib/python3.6/site-packages/pandas/core/frame.py in __getitem__(self, key)
   2684             return self._getitem_frame(key)
   2685         elif is_mi_columns:
-> 2686             return self._getitem_multilevel(key)
   2687         else:
   2688             return self._getitem_column(key)

~/Documents/AGCEC/MCastle/pymagicc/venv/lib/python3.6/site-packages/pandas/core/frame.py in _getitem_multilevel(self, key)
   2728 
   2729     def _getitem_multilevel(self, key):
-> 2730         loc = self.columns.get_loc(key)
   2731         if isinstance(loc, (slice, Series, np.ndarray, Index)):
   2732             new_columns = self.colum

In [10]:
# can also put index values in
mdata[1956, 'CO2I_EMIS', 'R5ASIA'].head(2)

VARIABLE,CO2I_EMIS
TODO,SET
UNITS,GtC
REGION,R5ASIA
YEAR,
1956,0.110425


In [11]:
%%expect_exception KeyError
# although slicing fails, one for the to do list...
mdata[1956:1970, 'CO2I_EMIS', 'R5ASIA'].head(2)

TypeError: (slice(1956, 1970, None), 'CO2I_EMIS', 'R5ASIA')

In [12]:
mdata[1998]

VARIABLE CO2I_EMIS                                        
TODO           SET                                        
UNITS          GtC                                        
REGION      R5OECD     R5REF    R5ASIA     R5MAF     R5LAM
YEAR                                                      
1998      3.184738  0.853781  1.648214  0.522509  0.378757

### Reading SCEN files

We can also read SCEN files. These are particularly tricky as the underlying data format is unique.

In [13]:
mdata.read(MAGICC6_DIR, "RCP45.SCEN")
mdata.df.head()

VARIABLE,CO2I,CO2B,CH4,N2O,SOX,CO,NMVOC,NOX,BC,OC,...,C6F14,HFC23,HFC32,HFC4310,HFC125,HFC134A,HFC143A,HFC227EA,HFC245FA,SF6
TODO,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET,...,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET
UNITS,GtC,GtC,MtCH4,MtN2O-N,MtS,MtCO,Mt,MtN,Mt,Mt,...,kt,kt,kt,kt,kt,kt,kt,kt,kt,kt
REGION,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,...,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS,BUNKERS
YEAR,,,,,,,,,,,,,,,,,,,,,
2000,6.735,1.1488,300.2070,7.4567,53.8413,1068.0009,210.6230,38.1623,7.8048,35.5434,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001,6.896,1.1320,303.4093,7.5029,54.4192,1066.7447,211.5938,38.2888,7.8946,35.7143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002,6.949,1.2317,306.5787,7.5487,54.9960,1065.4692,212.5632,38.4153,7.9841,35.8846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003,7.286,1.2256,309.7165,7.5942,55.5716,1064.1742,213.5311,38.5418,8.0734,36.0543,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2004,7.672,1.2428,312.8241,7.6394,56.1461,1062.8596,214.4977,38.6683,8.1624,36.2233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
mdata["BC", "R5ASIA"].head()

VARIABLE,BC
TODO,SET
UNITS,Mt
REGION,R5ASIA
YEAR,
2000,3.0468
2001,3.1090
2002,3.1818
2003,3.2546
2004,3.3274


## Writing files

Once you have your data in the format as above, writing files is trivial. 

In [15]:
mdata.read(MAGICC6_DIR, "HISTRCP_CO2I_EMIS.IN")
mdata.write("HISTEXAMPLE_CO2I_EMIS.IN")

mdata.read(MAGICC6_DIR, "RCP45.SCEN")
mdata.write("SCENEXAMPLE.SCEN", filepath='.')

However, note that the format to write in is determined by the filename. The rules for this are not yet documented (TODO!) but the errors are fairly clear. Hence you can't just any old filename, it has to follow MAGICC's (somewhat cryptic) internal conventions.

In [16]:
%%expect_exception ValueError
mdata.write("histexample.txt")

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-16-bd740518740c> in <module>()
----> 1 mdata.write("histexample.txt")

~/Documents/AGCEC/MCastle/pymagicc/pymagicc/input.py in write(self, filename, filepath)
    957         TODO: Implement writing to disk
    958         """
--> 959         writer = _get_writer(filename)
    960         writer.write(self, filename, filepath)

~/Documents/AGCEC/MCastle/pymagicc/pymagicc/input.py in _get_writer(fname)
    832 
    833 def _get_writer(fname):
--> 834     return determine_tool(fname, _fname_writer_regex_map, "Writer")()
    835 
    836 

~/Documents/AGCEC/MCastle/pymagicc/pymagicc/input.py in determine_tool(fname, regexp_map, tool_to_find)
    795             return regexp_map[fname_regex]
    796 
--> 797     raise ValueError("Couldn't find appropriate {} for {}".format(tool_to_find, fname))
    798 
    799 

ValueError: